# Projet MLOps - DPE

Voici le premier notebook du projet de construction d'un pipeline de modelsation predictive autour du dataset de l'[ADEME](https://www.ademe.fr/), le [DPE Tertiaire](https://data.ademe.fr/datasets/dpe-v2-tertiaire-2).

Ce notebook correpond au chapitre 2 du projet disponible sur [htts://skatai.com/mlops-dpe](htts://skatai.com/mlops-dpe).

SkatAI.com est un recueil de projets appliqués en IA et data science.

Le projet MLOps-DPE consiste a transformer un notebook classique de data science comprennant la transformation et numerisataion des données en un pipeline complet de production avec Airflow, MLFlow et FastAI.

Voici donc le premier notebook qui sert de point de depart a la suite du projet

Ce notebook est volontairement conçu comme etant assez simple et direct.

Les etapes sont les suivantes:

- charger les données a partir du site ADEME dans un dataframe pandas
- nettoyer les données : valeurs manquantes, aberrantes et surtout categorisation
- entrainer un modele random forest simple avec sckit-learn et gridsearch CV
- observer les metriques: precision, recall et score AUC ainsi que la matrice de classification
- sauver le modele une fois entrainé dans un fichier pickle


In [ ]:
# importer les librairies
import pandas as pd
import numpy as np
import re
import datetime
from collections import Counter

### Charger le dataset
À partir de l'url sur le site de l'ADEME

In [ ]:
# le fichier sur trouve a cette URL:
url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe-v2-tertiaire-2/lines?size=10000&format=csv&after=10000%2C965634&header=true"

# on le charge directement dans une dataframe Pandas
data = pd.read_csv(url)
print(data.shape)

(10000, 63)


Le dataset fait 10.000 echantillons mais comme nous verrons de nombreux n'ont pas d'etiquettes DPE et ne peuvent donc pas etre utilisé pour entrainer un modele.

## Exploration rapide
On va surtout s'attacher à remplacer les valeurs manquantes et a numeriser les categories.

Nous supprimons un grand nombre de variables qui pourraient se reveler tout a fait pertinentes mais dans le contexte du projet MLOP-DPE ne sont pas utiles. Entre autres les variables liées à la localisation des batiments.

In [ ]:
print(data.columns)

Index(['N°DPE', 'Date_réception_DPE', 'Date_établissement_DPE',
       'Date_visite_diagnostiqueur', 'Modèle_DPE', 'N°_DPE_remplacé',
       'Date_fin_validité_DPE', 'Version_DPE', 'N°_DPE_immeuble_associé',
       'Méthode_du_DPE', 'N°_immatriculation_copropriété',
       'Invariant_fiscal_logement', 'Etiquette_DPE', 'Etiquette_GES',
       'Conso_kWhep/m²/an', 'Emission_GES_kgCO2/m²/an', 'Année_construction',
       'Catégorie_ERP', 'Période_construction', 'Secteur_activité',
       'Nombre_occupant', 'Surface_(SHON)', 'Surface_utile',
       'Type_énergie_principale_chauffage', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence',
       'Complément_d'adresse_bâtiment', 'Cage_d'escalier',
       'Compléme

#### Renommer les colonnes

Les noms des variables sont partiiculierement alambiquées avec des signes degres, des parentheses, des / etc ... il vaut mieux standardirser tout cela et renommer les colonnes en minuscule avec un alphabet simplifé.

par exemple

- N°DPE -> n_dpe
- Conso_kWhep/m²/an -> conso_kwhep_m2_an
- N°_département_(BAN) -> n_departement_ban


On utilise une serie de regex pour transformer les noms dans la fonction _rename_columns_

In [ ]:
def rename_columns(columns):
    # en minuscule
    columns = [col.lower() for col in columns]
    # regex de remplacement
    rgxs = [
        (r"[°|/|']", "_"),
        (r"²", "2"),
        (r"[(|)]", ""),
        (r"é|è", "e"),
        (r"_+", "_"),
    ]
    # on remplace toutes les colonnes une par une
    for rgx in rgxs:
        columns = [re.sub(rgx[0], rgx[1], col) for col in columns]

    return columns


In [ ]:
data.columns = rename_columns(data.columns)

ce qui donne maintenant des noms de colonnes bien plus simples

In [ ]:
data.columns

Index(['n_dpe', 'date_reception_dpe', 'date_etablissement_dpe',
       'date_visite_diagnostiqueur', 'modele_dpe', 'n_dpe_remplace',
       'date_fin_validite_dpe', 'version_dpe', 'n_dpe_immeuble_associe',
       'methode_du_dpe', 'n_immatriculation_copropriete',
       'invariant_fiscal_logement', 'etiquette_dpe', 'etiquette_ges',
       'conso_kwhep_m2_an', 'emission_ges_kgco2_m2_an', 'annee_construction',
       'categorie_erp', 'periode_construction', 'secteur_activite',
       'nombre_occupant', 'surface_shon', 'surface_utile',
       'type_energie_principale_chauffage', 'adresse_brute', 'nom_commune_ban',
       'code_insee_ban', 'n_voie_ban', 'identifiant_ban', 'adresse_ban',
       'code_postal_ban', 'score_ban', 'nom_rue_ban',
       'coordonnee_cartographique_x_ban', 'coordonnee_cartographique_y_ban',
       'code_postal_brut', 'n_etage_appartement', 'nom_residence',
       'complement_d_adresse_bâtiment', 'cage_d_escalier',
       'complement_d_adresse_logement', 'statut_geo

### Les valeurs manquantes
Traitons tout d'abord les valeurs de la cible, puis les valeurs numeriqes (float et int)


In [ ]:
target = "etiquette_dpe"
data.dropna(subset=target, inplace=True)


Il reste beaucoup moins d'echantillons (~55%)

In [ ]:
data.shape

(5553, 63)

#### Les valeurs floats et int

On defini les colonnes floats et les colonnes int

Toutes le valeurs numeriques sont positives. On va remplaces les valeurs manquantes par un -1.0 et les valeurs int par un -1

Il n'y a pas vraiment besoin d'avoir une grande precision p[our les valeusr floats.
On va donc tout convertir en type int

Mais definissons d'abord les colonnes floats puis et int

In [ ]:
columns_float = [
    "conso_kwhep_m2_an",
    "emission_ges_kgco2_m2_an",
    "surface_utile",
    "conso_e_finale_energie_n_1",
    "conso_e_primaire_energie_n_1",
    "frais_annuel_energie_n_1",
    "conso_e_finale_energie_n_2",
    "conso_e_primaire_energie_n_2",
    "frais_annuel_energie_n_2",
    "conso_e_finale_energie_n_3",
    "conso_e_primaire_energie_n_3",
    "frais_annuel_energie_n_3",
]

for col in columns_float:
    data[col].fillna(-1.0, inplace=True)
    data[col] = data[col].astype(int)


In [ ]:
# puis les int
columns_int = [
    "annee_construction",
    "nombre_occupant",
    "n_etage_appartement",
]

for col in columns_int:
    data[col].fillna(-1, inplace=True)
    data[col] = data[col].astype(int)


In [ ]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5553 entries, 4 to 9999
Data columns (total 63 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   n_dpe                              5553 non-null   object 
 1   date_reception_dpe                 5553 non-null   object 
 2   date_etablissement_dpe             5553 non-null   object 
 3   date_visite_diagnostiqueur         5553 non-null   object 
 4   modele_dpe                         5553 non-null   object 
 5   n_dpe_remplace                     440 non-null    object 
 6   date_fin_validite_dpe              5553 non-null   object 
 7   version_dpe                        5553 non-null   float64
 8   n_dpe_immeuble_associe             24 non-null     object 
 9   methode_du_dpe                     5292 non-null   object 
 10  n_immatriculation_copropriete      16 non-null     object 
 11  invariant_fiscal_logement          18 non-null     object 
 1

### Ne garder qu'un minimum de colonnes

Voici les colonnes que nous allons garder au final
libre a vous de modifier cette liste si vous preferez.




In [ ]:
columns_categorical = [
    "periode_construction",
    "secteur_activite",
    "type_energie_principale_chauffage",
    "type_energie_n_1",
    "type_usage_energie_n_1",
]

columns_num = [
    "version_dpe",
    "surface_utile",
    "conso_kwhep_m2_an",
    "conso_e_finale_energie_n_1",
    "version_dpe",
]

train_columns = [
    "version_dpe",
    "periode_construction",
    "secteur_activite",
    "type_energie_principale_chauffage",
    "type_energie_n_1",
    "type_usage_energie_n_1",
    "surface_utile",
    "conso_kwhep_m2_an",
    "conso_e_finale_energie_n_1",
]


target = 'etiquette_dpe'
id = 'n_dpe'

## Valeurs manquantes

Par convention les valeurs manquates categoriques sont mises a "" et les float et int ont la valeur -1


In [ ]:
for col in columns_categorical:
    data[col].fillna("", inplace=True)

for col in columns_num:
    data[col].fillna(-1, inplace=True)
    data.loc[data[col] == "", col] = -1.0



### Colonnes categoriques

Il y a plein de methodes pour encoder des valeurs numeriques: ont hot encoding, ordinal encoding, binary encoding

La librairie [binary encoder](https://pypi.org/project/category-encoders/) offre de nombreuses methodes.

Dans notre contexte, on va simplement assigner une valeur entiere par categorie. Ouh làlà vous mme direz ?! Cela va rajoutez un information dans les données en ordonnant les valeurs alors qu'elles en sont pas ordonnées initialement (non ordinale)

En effet. cependant dans mon experience cela ne changera pas grand chose par rapport au d'autres methodes comme le one hot encoding ou binary encoder. par contre en va gagner en siimplicité.

Cependant pour que l'ordre ai un sens nous pouvons classer les categories non pas cpar ordre alphabetique mais par ordre de leur frequence dans les données. cela nou permettra de regrouper les valeurs moins frequentes (la longue **tail** des data) ensemble et de les regrouper ensemble.


Pour cela nous allons simplement regarder la frequence des categories avec _Counter_ pour les différentes valeurs pour chqaue categorie et ecrire a la main le dictionnaire de conversion

In [ ]:
def frequence(col):
    categories = Counter(data[col])
    print(categories.most_common())


Ce qui donne pour les categories principales : type energie et type usage.

Par exemple Electricité > Gaz naturel > Chauffage Urbain etc

ou Chauffage > Eau chaude > Fioul etc


Nous vous laissons faire autres

In [ ]:
frequence('type_energie_principale_chauffage')

[('', 4511), ('Électricité', 705), ('Gaz naturel', 252), ('Réseau de Chauffage urbain', 33), ('Fioul domestique', 24), ('autre combustible fossile', 21), ('Bois – Granulés (pellets) ou briquettes', 4), ('Bois – Bûches', 1), ('GPL', 1), ('Propane', 1)]


In [ ]:
frequence('type_usage_energie_n_1')

[("périmètre de l'usage inconnu", 4156), ('Chauffage', 900), ('Eau Chaude sanitaire', 208), ('Eclairage', 89), ('Ascenseur(s)', 58), ('', 49), ('Autres usages', 42), ('Refroidissement', 31), ('auxiliaires et ventilation', 8), ('Bureautique', 7), ("Production d'électricité à demeure", 5)]


On peut alors definir le dictionnaire suivant pour le type d'energie qui concernera les variables : type_energie_principale_chauffage, type_energie_n_1, type_energie_n_2, type_energie_n_3


In [ ]:
map_type_energie = {
    "non renseigné": -1,
    "Électricité": 1,
    "Électricité d'origine renouvelable utilisée dans le bâtiment": 1,
    "Gaz naturel": 2,
    "Butane": 2,
    "Propane": 2,
    "GPL": 2,
    "Fioul domestique": 3,
    "Réseau de Chauffage urbain": 4,
    "Charbon": 5,
    "autre combustible fossile": 5,
    "Bois – Bûches": 6,
    "Bois – Plaquettes forestières": 6,
    "Bois – Granulés (pellets) ou briquettes": 6,
    "Bois – Plaquettes d’industrie": 6,
}

Pour l'usage cela concernera les variables : type_usage_energie_n_1, type_usage_energie_n_2, type_usage_energie_n_3



In [ ]:
# Pour l'usage

map_type_usage = {
    "non renseigné": -1,
    "périmètre de l'usage inconnu": -1,
    "Chauffage": 1,
    "Eau Chaude sanitaire": 2,
    "Eclairage": 3,
    "Refroidissement": 4,
    "auxiliaires et ventilation": 4,
    "Ascenseur(s)": 5,
    "Autres usages": 6,
    "Bureautique": 6,
    "Abonnements": 6,
    "Production d'électricité à demeure": 6,
}


et pour les autres variables categoriques que nous garderons :categorie_erp et secteur_activite:


In [ ]:
map_secteur_activite = {
    "autres tertiaires non ERP": 1,
    "M : Magasins de vente, centres commerciaux": 2,
    "W : Administrations, banques, bureaux": 3,
    "locaux d'entreprise (bureaux)": 4,
    "J : Structures d’accueil pour personnes âgées ou personnes handicapées": 5,
    "N : Restaurants et débits de boisson": 6,
    "U : Établissements de soins": 7,
    "GHW : Bureaux": 8,
    "R : Établissements d’éveil, d’enseignement, de formation, centres de vacances, centres de loisirs sans hébergement": 9,
    "O : Hôtels et pensions de famille": 10,
    "GHZ : Usage mixte": 11,
    "X : Établissements sportifs couverts": 12,
    "L : Salles d'auditions, de conférences, de réunions, de spectacles ou à usage multiple": 13,
    "T : Salles d'exposition à vocation commerciale": 14,
    "P : Salles de danse et salles de jeux": 15,
    "GHR : Enseignement": 16,
    "V : Établissements de divers cultes": 17,
    "S : Bibliothèques, centres de documentation": 18,
    "OA : Hôtels-restaurants d'Altitude": 19,
    "GHU : Usage sanitaire": 20,
    "PA : Établissements de Plein Air": 21,
    "GHA : Habitation": 22,
    "GHO : Hôtel": 23,
    "Y : Musées": 24,
    "PS : Parcs de Stationnement couverts": 25,
    "GHTC : tour de contrôle": 26,
    "REF : REFuges de montagne": 27,
    "GA : Gares Accessibles au public (chemins de fer, téléphériques, remonte-pentes...)": 28,
    "CTS : Chapiteaux, Tentes et Structures toile": 29,
    "GHS : Dépôt d'archives": 30,
}


In [ ]:
map_categorie_erp = {
    "1ère Catégorie": 1,
    "2ème Catégorie": 2,
    "3ème Catégorie": 3,
    "4ème Catégorie": 4,
    "5ème Catégorie": 5,
}


Et pour finir la periode de construction


In [ ]:
map_periode_construction = {
    "avant 1948": 0,
    "1948-1974": 1,
    "1975-1977": 2,
    "1978-1982": 3,
    "1983-1988": 4,
    "1989-2000": 5,
    "2001-2005": 6,
    "2006-2012": 7,
    "2013-2021": 8,
    "après 2021": 9,
}


Pour la variable cible, la conversion est directe

In [ ]:
map_target = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7}

Pour convertir ensuite les variables il suffit d'appliquer les dictionnaires aux bonnes colonnes

In [ ]:
def encode_categorical_with_map(data, column, mapping, default_unknown=""):
    valid_values = list(mapping.keys())
    # id unknown values
    data.loc[data[column].isin(valid_values), column] = default_unknown
    # always cast missing values as -1
    mapping[default_unknown] = -1
    # encode
    data[column] = data[column].apply(lambda d: mapping[d])
    return data[col]


Et donc nous avons

In [ ]:
mappings = [map_periode_construction,map_secteur_activite, map_type_energie, map_type_energie ,map_type_usage]


for col, mapping in zip(columns_categorical, mappings):
    data[col] = encode_categorical_with_map(data, col, mapping)

In [ ]:
data[target] = encode_categorical_with_map(data, target, map_target)

La dataframe est maintenant entierement numerisé aussi bien pour les colonnes d'entrainement que pour la variable cible. Il n;y a que l'ID: "n_ndp" qui reste comme string.

Dans la suite du projet nous gardons cette ID _au cas ou_

In [ ]:
data[train_columns].head()

,version_dpe,periode_construction,secteur_activite,type_energie_principale_chauffage,type_energie_n_1,type_usage_energie_n_1,surface_utile,conso_kwhep_m2_an,conso_e_finale_energie_n_1
4,2.2,-1,-1,-1,-1,-1,100,186,10684
6,1.0,-1,-1,-1,-1,-1,110,159,7622
9,2.3,-1,-1,-1,-1,-1,100,281,16887
10,2.1,-1,-1,-1,-1,-1,100,230,18930
13,2.1,-1,-1,-1,-1,-1,100,58,3197


In [ ]:
data[target].value_counts()

etiquette_dpe
-1    5553
Name: count, dtype: int64

### Conclusion
les données sont ...
- P features
- une variable cible
- N echantillons

# Training

Passons maitenant a la phase d'entrainement du modele. On importe les librairies necessaires de scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

In [ ]:
# # load data
# input_file = output_file
# data = pd.read_csv(input_file)
# # shuffle
# data = data.sample(frac=1, random_state=808).reset_index(drop=True)


comme d'habitude on split le ataset en une partie d'entrainement et une de test

In [ ]:
# Assuming the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]  # Target variable
assert y.name == "etiquette_dpe"

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=808
)

X_train.drop(columns=["n_dpe"], inplace=True)
id_test = list(X_test.n_dpe)
X_test.drop(columns=["n_dpe"], inplace=True)


In [ ]:
    # Initialize the model
    rf = RandomForestClassifier()


In [ ]:
    # Define the parameter grid
    param_grid = {
        "n_estimators": [200, 300],  # Number of trees
        "max_depth": [10],  # Maximum depth of the trees
        "min_samples_leaf": [1, 5],  # Maximum depth of the trees
    }

    # Setup GridSearchCV with k-fold cross-validation
    cv = KFold(n_splits=3, random_state=84, shuffle=True)

    grid_search = GridSearchCV(
        estimator=rf, param_grid=param_grid, cv=cv, scoring="accuracy", verbose=1
    )


In [ ]:
    # Fit the model
    grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=KFold(n_splits=3, random_state=84, shuffle=True),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10], 'min_samples_leaf': [1, 5],
                         'n_estimators': [200, 300]},
             scoring='accuracy', verbose=1)

In [ ]:
    # Best parameters and best score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_}")
    print(f"Best model: {grid_search.best_estimator_}")

    # Evaluate on the test set
    yhat = grid_search.predict(X_test)
    print(classification_report(y_test, yhat))



Best parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 300}
Best cross-validation score: 0.8218103181958604
Best model: RandomForestClassifier(max_depth=10, n_estimators=300)
              precision    recall  f1-score   support

           1       0.99      0.87      0.92       105
           2       0.83      0.86      0.84       121
           3       0.93      0.82      0.87       291
           4       0.82      0.85      0.83       270
           5       0.69      0.89      0.78       147
           6       0.90      0.72      0.80        64
           7       0.89      0.90      0.90       124

    accuracy                           0.85      1122
   macro avg       0.86      0.84      0.85      1122
weighted avg       0.86      0.85      0.85      1122



In [ ]:
# regroup into predictions dataframe
probabilities = grid_search.predict_proba(X_test)

predictions = pd.DataFrame()
predictions["id"] = id_test
predictions["prob"] = np.max(probabilities, axis=1)
predictions["yhat"] = yhat
predictions["y"] = y_test.values
print(predictions.head())


              id   prob  yhat  y
0  2113T0707016A 1.0000     1  1
1  2373T4274354S 0.7016     4  4
2  2142T1010760P 0.4683     3  3
3  2244T0873273T 0.4456     5  4
4  2294T1585219F 0.9323     3  3


In [ ]:
    # feature importance
    feature_importances = grid_search.best_estimator_.feature_importances_
    feature_names = X_train.columns

    # Create a dictionary mapping feature names to their importance
    importance_dict = dict(zip(feature_names, feature_importances))
    importance_dict = dict(
        sorted(importance_dict.items(), key=lambda item: item[1], reverse=True)
    )

    print(importance_dict)



{'conso_kwhep_m2_an': 0.4549981118085486, 'emission_ges_kgco2_m2_an': 0.2297529530166314, 'conso_finale_energie': 0.06971622227648284, 'conso_primaire_energie': 0.06810497358739391, 'frais_annuel_energie': 0.04892797667974759, 'secteur_activite': 0.029974549662307958, 'surface_utile': 0.01973088010602134, 'annee_construction': 0.01712967254261392, 'methode_du_dpe': 0.015321822056583048, 'version_dpe': 0.013908306206751703, 'type_energie_n_1': 0.013734412569784388, 'categorie_erp': 0.007965915746066369, 'type_usage_energie_n_1': 0.005574588670296643, 'type_energie_principale_chauffage': 0.005159615070770455}
